### ML notebook

In [112]:
n_samples = 1000
gens_per_ideal = 2

In [113]:
# P.<x,y> = PolynomialRing(QQ, 2, order='deglex')
P.<x,y> = PolynomialRing(QQ, 2, order='lex')
# P.<x,y> = PolynomialRing(QQ, 2, order='degrevlex')
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('wdegrevlex',(1,3)))
# P.<x,y> = PolynomialRing(QQ, 2, order=TermOrder('negwdegrevlex',(1,3)))

#### Functions to create features and label

In [114]:
def is_GB_worth(GB): # Receives a GB
    n_depend1=0
    n_depend2=0
    for pol in GB:
        depend_x=False
        depend_y=False
        
        # Comprobamos si depende de x y de y
        if pol.degree(x) > 0:
            depend_x=True
        if pol.degree(y) > 0:
            depend_y=True
            
        # Actualizamos el numero de de polinomios que dependen de cada cosa
        if depend_x and depend_y:
            n_depend2 += 1
        elif depend_x or depend_y:
            n_depend1 += 1
    if n_depend1 >= n_depend2:
        return True
    else:
        return False
    
def terms_on_x(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[0] > 0:
                count+=1
    return count
        
def terms_on_y(gens):
    count=0
    for item in gens:
        for exp in item.exponents():
            if exp[1] > 0:
                count+=1
    return count

#### Voy a generar una lista de ideales para los polinomios generados aleatoriamente y con información adicional

In [115]:
lstOfIdeals = []

for i in range(n_samples):
    generators = []
    n_homogeneous = 0
    
    for j in range(gens_per_ideal):
        p = P.random_element()
        generators.append(p)
        if p.is_homogeneous():
            n_homogeneous += 1
    # Generadores del ideal en una lista
        generators.append(p)
    
    # Definimos el ideal a partir de los generadores anteriores
    I = ideal(generators)
    B = I.groebner_basis()
    lstOfIdeals.append([I, n_homogeneous, terms_on_x(generators), terms_on_y(generators), is_GB_worth(B)])

# Conjunto de polinomios generado de manera aleatoria
lstOfIdeals[:3]

[[Ideal (-2*x*y - 1/23*x - 12*y^2 + 2, -2*x*y - 1/23*x - 12*y^2 + 2, -558*x^2 - 6*x + 1/3*y^2 - 1, -558*x^2 - 6*x + 1/3*y^2 - 1) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  8,
  6,
  True],
 [Ideal (3/2*x^2 - 11*x - 39*y^2 - y, 3/2*x^2 - 11*x - 39*y^2 - y, -9*x^2 + 76/29*x*y - y^2 - y + 1, -9*x^2 + 76/29*x*y - y^2 - y + 1) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  8,
  10,
  True],
 [Ideal (2*x*y + 2*y - 1, 2*x*y + 2*y - 1, 3/19*x^2 + 1/5*x*y - 1/5*y^2 + 1/2*y - 1, 3/19*x^2 + 1/5*x*y - 1/5*y^2 + 1/2*y - 1) of Multivariate Polynomial Ring in x, y over Rational Field,
  0,
  6,
  10,
  True]]

In [116]:
!sage --pip install numpy scipy sklearn pandas

#### Definimos las features

In [117]:
import numpy as np

X = np.array([[item[1],item[2],item[3]] for item in lstOfIdeals])
y = np.array([item[4] for item in lstOfIdeals])

In [118]:
X[:5]

array([[ 0,  8,  6],
       [ 0,  8, 10],
       [ 0,  6, 10],
       [ 0,  6,  6],
       [ 0, 12,  8]])

In [119]:
y[:5]

array([ True,  True,  True,  True,  True])

In [120]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [121]:
from sklearn import svm

classifier = svm.SVC()

In [122]:
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [123]:
preds = classifier.predict(X_test)

In [124]:
hits=0
fails=0
for i in range(len(y_test)):
    if y_test[i] == preds[i]:
        hits+=1
    else:
        fails+=1

In [125]:
print('Hits: ' + str(hits), 'Fails: '+str(fails))

('Hits: 317', 'Fails: 13')
